## オプション情報だけでも拾いたい

ページ情報を拾う
    分離可能
→ページのURLから詳細情報を拾う
    分離可能
→オプション情報を拾う

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
import time
from retry import retry
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

In [3]:
# 音を出す
def sound():
    !rundll32 user32.dll,MessageBeep

In [4]:
# 検索ページのurlから物件の詳細を拾ってきたい
# ここはurl情報だけ取ってくる

# 情報の器
detail_urls = []

# 検索出力ページ（東京23区）
base_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'

# 拾うページ数
max_page = 2000

# ページ読み込み関数（リトライ機能付き）
@retry(tries=3, delay=10, backoff=2)
def load_page(load_url):
    html = requests.get(load_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []

# ページごとの処理
for page in tqdm(range(1,max_page+1)):
    before = time.time()

    # ページ遷移
    load_url = base_url.format(page)
    # 検索ページ情報
    soup = load_page(load_url)

    # 検索ページから物件詳細の絶対urlを獲得
    for j, element in enumerate(soup.find_all(class_='js-cassette_link_href cassetteitem_other-linktext')):
        get_url = element.get('href')
        abs_url = urllib.parse.urljoin(load_url,get_url)
#         soup_room = load_page(abs_url)

        # 部屋情報の器
        data_room = []

#         # 物件名
#         name = soup_room.find(class_='section_h1-header-title').text
#         data_room.append(name)
#         # 費用に関する情報
#         colname = ['家賃','管理費・共益費','敷金','礼金','保証金','敷引・償却']
#         cost_info = soup_room.find_all(class_='property_view_note-list')
#         for mother in cost_info:
#             for child in mother.find_all('span'):
#                 data_room.append(child.text)
#         # 基本情報
#         colname = ['住所','駅徒歩','間取り','面積','築年数','階','向き','建物種別']
#         basic_info = soup_room.find_all(class_='property_view_table-body')
#         for k, (name, info ) in enumerate(zip(colname,basic_info)):
#             if k == 1:
#                 for child in info.find_all('div'):
#                     data_room.append(child.text)
#             else:
#                 data_room.append(info.text)
#         # オプション情報
#         option = soup_room.find_all(class_='inline_list')
#         data_room.append(option[1].text)
#         # 概要情報
#         colname = ['間取り詳細','構造','階建','築年月','損保','駐車場','入居','取引態様','条件','取り扱い店舗物件コード','SUUMO物件コード','総戸数','情報更新日','次回更新日','仲介手数料','保証会社','ほか初期費用']
#         overview = soup_room.find(class_='data_table table_gaiyou')
#         children = overview.find_all('td')
#         for name, info in zip(colname,children):
#             data_room.append(info.text.strip())
            
#        # url情報
#        data_room.append(abs_url)

        # 器に部屋情報を入れる
        detail_urls.append(abs_url)

    # requestsしたら1秒休む
    time.sleep(1)
        
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(detail_urls)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)
sound()

  0%|          | 0/2000 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [98]:
# urls = pd.DataFrame(detail_urls,columns=['url'])
# urls.to_csv('detail_urls.csv',index=False)

In [5]:
urls = pd.read_csv('detail_urls.csv')
detail_urls = urls['url'].tolist()

In [6]:
# 検索ページのurlから物件の詳細を拾ってきたい
# url情報を使って物件ごとのsoupを作る

detail_soups = []

# ページ読み込み関数（リトライ機能付き）
@retry(tries=3, delay=10, backoff=2)
def load_page(load_url):
    html = requests.get(load_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
times = []
times.append(time.time())
runtimes = []

# 途中から始めるときの調整
finish_num = 129000
urls_from_the_middle = detail_urls[finish_num:]

# urlごとの処理
for num, url in enumerate(urls_from_the_middle):

    # soupを収納する
    soup_room = load_page(url)
    detail_soups.append([url,soup_room])

    # requestsしたら1秒休む
    time.sleep(1)
    
    # 100soupごとにcsvに書き出す
    if (num+1) % 100 == 0 :
        soup_tank = pd.DataFrame(detail_soups, columns=['url','soup'])
        soup_tank.to_csv(f'./soup_tank/soup_tank_{finish_num+num+1}.csv',index=False)
        detail_soups = []
        
        # 進捗確認
        # このページの作業時間を表示
        times.append(time.time())
        running_time = times[-1] - times[-2]
        runtimes.append(running_time)
        print(f'{(num+1)/100}ページ目：{running_time}秒')
        # 取得した件数
        print(f'総取得件数：{num+1}')
        # 作業進捗
        complete_ratio = round((num+1)/len(urls_from_the_middle)*100,3)
        print(f'完了：{complete_ratio}%')
        # 作業の残り時間目安を表示
        running_mean = np.mean(runtimes)
        running_required_time = running_mean * (int((len(urls_from_the_middle) - (num+1))/100)+1)
        hour = int(running_required_time/3600)
        minute = int((running_required_time%3600)/60)
        second = int(running_required_time%60)
        print(f'残り時間：{hour}時間{minute}分{second}秒\n')
        sound()

# 処理時間を測りたい
running_all = times[-1] - times[0]
print('総経過時間：',running_all)
sound()

1.0ページ目：172.31522679328918秒
総取得件数：100
完了：0.106%
残り時間：45時間19分42秒

2.0ページ目：167.80467677116394秒
総取得件数：200
完了：0.211%
残り時間：44時間41分16秒

3.0ページ目：162.38859391212463秒
総取得件数：300
完了：0.317%
残り時間：43時間58分10秒

4.0ページ目：161.81138467788696秒
総取得件数：400
完了：0.422%
残り時間：43時間32分59秒

5.0ページ目：184.21024417877197秒
総取得件数：500
完了：0.528%
残り時間：44時間27分12秒

6.0ページ目：179.38208413124084秒
総取得件数：600
完了：0.633%
残り時間：44時間49分42秒

7.0ページ目：174.14803194999695秒
総取得件数：700
完了：0.739%
残り時間：44時間53分11秒

8.0ページ目：175.0299289226532秒
総取得件数：800
完了：0.845%
残り時間：44時間56分48秒

9.0ページ目：173.81965231895447秒
総取得件数：900
完了：0.95%
残り時間：44時間56分51秒

10.0ページ目：168.22977352142334秒
総取得件数：1000
完了：1.056%
残り時間：44時間47分35秒

11.0ページ目：179.6036446094513秒
総取得件数：1100
完了：1.161%
残り時間：44時間55分38秒

12.0ページ目：172.41676712036133秒
総取得件数：1200
完了：1.267%
残り時間：44時間52分30秒

13.0ページ目：170.65024614334106秒
総取得件数：1300
完了：1.372%
残り時間：44時間47分17秒

14.0ページ目：174.86991000175476秒
総取得件数：1400
完了：1.478%
残り時間：44時間47分7秒

15.0ページ目：202.21699905395508秒
総取得件数：1500
完了：1.584%
残り時間：45時間14分55秒

16.0ページ目：164.1832

124.0ページ目：157.6252908706665秒
総取得件数：12400
完了：13.091%
残り時間：39時間46分43秒

125.0ページ目：161.00879955291748秒
総取得件数：12500
完了：13.197%
残り時間：39時間42分25秒

126.0ページ目：158.88856720924377秒
総取得件数：12600
完了：13.302%
残り時間：39時間37分55秒

127.0ページ目：177.0100154876709秒
総取得件数：12700
完了：13.408%
残り時間：39時間35分24秒

128.0ページ目：163.91420030593872秒
総取得件数：12800
完了：13.513%
残り時間：39時間31分28秒

129.0ページ目：172.95458126068115秒
総取得件数：12900
完了：13.619%
残り時間：39時間28分31秒

130.0ページ目：160.51211714744568秒
総取得件数：13000
完了：13.725%
残り時間：39時間24分16秒

131.0ページ目：159.05606126785278秒
総取得件数：13100
完了：13.83%
残り時間：39時間19分53秒

132.0ページ目：184.68609404563904秒
総取得件数：13200
完了：13.936%
残り時間：39時間18分10秒

133.0ページ目：166.2076621055603秒
総取得件数：13300
完了：14.041%
残り時間：39時間14分32秒

134.0ページ目：170.26594972610474秒
総取得件数：13400
完了：14.147%
残り時間：39時間11分20秒

135.0ページ目：177.8354730606079秒
総取得件数：13500
完了：14.252%
残り時間：39時間8分54秒

136.0ページ目：170.93784070014954秒
総取得件数：13600
完了：14.358%
残り時間：39時間5分46秒

137.0ページ目：156.10899424552917秒
総取得件数：13700
完了：14.464%
残り時間：39時間1分11秒

138.0ページ目：166.1442472934723秒

243.0ページ目：159.24356341362秒
総取得件数：24300
完了：25.654%
残り時間：33時間2分47秒

244.0ページ目：154.0089933872223秒
総取得件数：24400
完了：25.76%
残り時間：32時間59分16秒

245.0ページ目：165.68996858596802秒
総取得件数：24500
完了：25.865%
残り時間：32時間56分18秒

246.0ページ目：154.54629135131836秒
総取得件数：24600
完了：25.971%
残り時間：32時間52分49秒

247.0ページ目：163.01249289512634秒
総取得件数：24700
完了：26.077%
残り時間：32時間49分45秒

248.0ページ目：174.74521684646606秒
総取得件数：24800
完了：26.182%
残り時間：32時間47分13秒

249.0ページ目：152.57616806030273秒
総取得件数：24900
完了：26.288%
残り時間：32時間43分40秒

250.0ページ目：175.0625307559967秒
総取得件数：25000
完了：26.393%
残り時間：32時間41分9秒

251.0ページ目：152.73527789115906秒
総取得件数：25100
完了：26.499%
残り時間：32時間37分37秒

252.0ページ目：175.13159942626953秒
総取得件数：25200
完了：26.604%
残り時間：32時間35分7秒

253.0ページ目：157.87411761283875秒
総取得件数：25300
完了：26.71%
残り時間：32時間31分49秒

254.0ページ目：176.35086703300476秒
総取得件数：25400
完了：26.816%
残り時間：32時間29分22秒

255.0ページ目：183.02046394348145秒
総取得件数：25500
完了：26.921%
残り時間：32時間27分12秒

256.0ページ目：166.63310980796814秒
総取得件数：25600
完了：27.027%
残り時間：32時間24分19秒

257.0ページ目：172.67470908164978秒


362.0ページ目：158.28487253189087秒
総取得件数：36200
完了：38.218%
残り時間：27時間20分38秒

363.0ページ目：158.5851607322693秒
総取得件数：36300
完了：38.323%
残り時間：27時間17分35秒

364.0ページ目：161.03007555007935秒
総取得件数：36400
完了：38.429%
残り時間：27時間14分36秒

365.0ページ目：161.93987679481506秒
総取得件数：36500
完了：38.534%
残り時間：27時間11分38秒

366.0ページ目：168.32021284103394秒
総取得件数：36600
完了：38.64%
残り時間：27時間8分51秒

367.0ページ目：202.41986918449402秒
総取得件数：36700
完了：38.745%
残り時間：27時間6分58秒

368.0ページ目：162.87005734443665秒
総取得件数：36800
完了：38.851%
残り時間：27時間4分2秒

369.0ページ目：179.0800883769989秒
総取得件数：36900
完了：38.957%
残り時間：27時間1分31秒

370.0ページ目：174.9799952507019秒
総取得件数：37000
完了：39.062%
残り時間：26時間58分54秒

371.0ページ目：181.0900478363037秒
総取得件数：37100
完了：39.168%
残り時間：26時間56分26秒

372.0ページ目：167.91091561317444秒
総取得件数：37200
完了：39.273%
残り時間：26時間53分38秒

373.0ページ目：176.10883045196533秒
総取得件数：37300
完了：39.379%
残り時間：26時間51分2秒

374.0ページ目：166.62023401260376秒
総取得件数：37400
完了：39.484%
残り時間：26時間48分12秒

375.0ページ目：168.5947778224945秒
総取得件数：37500
完了：39.59%
残り時間：26時間45分24秒

376.0ページ目：167.34504103660583秒
総取得

481.0ページ目：182.31883311271667秒
総取得件数：48100
完了：50.781%
残り時間：21時間59分14秒

482.0ページ目：177.1296124458313秒
総取得件数：48200
完了：50.886%
残り時間：21時間56分32秒

483.0ページ目：192.14339184761047秒
総取得件数：48300
完了：50.992%
残り時間：21時間54分4秒

484.0ページ目：177.30123782157898秒
総取得件数：48400
完了：51.097%
残り時間：21時間51分22秒

485.0ページ目：180.402578830719秒
総取得件数：48500
完了：51.203%
残り時間：21時間48分43秒

486.0ページ目：180.36340880393982秒
総取得件数：48600
完了：51.309%
残り時間：21時間46分4秒

487.0ページ目：183.45945119857788秒
総取得件数：48700
完了：51.414%
残り時間：21時間43分27秒

488.0ページ目：186.1338188648224秒
総取得件数：48800
完了：51.52%
残り時間：21時間40分53秒

489.0ページ目：169.8189353942871秒
総取得件数：48900
完了：51.625%
残り時間：21時間38分3秒

490.0ページ目：169.71492648124695秒
総取得件数：49000
完了：51.731%
残り時間：21時間35分14秒

491.0ページ目：181.35877394676208秒
総取得件数：49100
完了：51.836%
残り時間：21時間32分35秒

492.0ページ目：160.38814282417297秒
総取得件数：49200
完了：51.942%
残り時間：21時間29分37秒

493.0ページ目：172.67798733711243秒
総取得件数：49300
完了：52.048%
残り時間：21時間26分50秒

494.0ページ目：176.97579741477966秒
総取得件数：49400
完了：52.153%
残り時間：21時間24分7秒

495.0ページ目：180.55384850502014秒


600.0ページ目：151.90744376182556秒
総取得件数：60000
完了：63.344%
残り時間：16時間28分45秒

601.0ページ目：156.06282544136047秒
総取得件数：60100
完了：63.449%
残り時間：16時間25分46秒

602.0ページ目：167.32742261886597秒
総取得件数：60200
完了：63.555%
残り時間：16時間22分54秒

603.0ページ目：171.47238969802856秒
総取得件数：60300
完了：63.661%
残り時間：16時間20分4秒

604.0ページ目：160.64315056800842秒
総取得件数：60400
完了：63.766%
残り時間：16時間17分8秒

605.0ページ目：185.32259249687195秒
総取得件数：60500
完了：63.872%
残り時間：16時間14分26秒

606.0ページ目：179.63210201263428秒
総取得件数：60600
完了：63.977%
残り時間：16時間11分41秒

607.0ページ目：153.56729674339294秒
総取得件数：60700
完了：64.083%
残り時間：16時間8分41秒

608.0ページ目：166.6294596195221秒
総取得件数：60800
完了：64.189%
残り時間：16時間5分48秒

609.0ページ目：173.02557492256165秒
総取得件数：60900
完了：64.294%
残り時間：16時間2分59秒

610.0ページ目：164.99128007888794秒
総取得件数：61000
完了：64.4%
残り時間：16時間0分6秒

611.0ページ目：167.4289321899414秒
総取得件数：61100
完了：64.505%
残り時間：15時間57分14秒

612.0ページ目：161.93054580688477秒
総取得件数：61200
完了：64.611%
残り時間：15時間54分19秒

613.0ページ目：177.6546447277069秒
総取得件数：61300
完了：64.716%
残り時間：15時間51分32秒

614.0ページ目：167.3044855594635秒
総取得

719.0ページ目：177.8789095878601秒
総取得件数：71900
完了：75.907%
残り時間：10時間48分49秒

720.0ページ目：153.19437766075134秒
総取得件数：72000
完了：76.013%
残り時間：10時間45分54秒

721.0ページ目：172.46091866493225秒
総取得件数：72100
完了：76.118%
残り時間：10時間43分5秒

722.0ページ目：172.14710807800293秒
総取得件数：72200
完了：76.224%
残り時間：10時間40分15秒

723.0ページ目：162.59330010414124秒
総取得件数：72300
完了：76.329%
残り時間：10時間37分23秒

724.0ページ目：156.65515995025635秒
総取得件数：72400
完了：76.435%
残り時間：10時間34分29秒

725.0ページ目：170.04181456565857秒
総取得件数：72500
完了：76.541%
残り時間：10時間31分39秒

726.0ページ目：161.86831259727478秒
総取得件数：72600
完了：76.646%
残り時間：10時間28分47秒

727.0ページ目：158.5994997024536秒
総取得件数：72700
完了：76.752%
残り時間：10時間25分53秒

728.0ページ目：159.57775783538818秒
総取得件数：72800
完了：76.857%
残り時間：10時間23分0秒

729.0ページ目：168.38383507728577秒
総取得件数：72900
完了：76.963%
残り時間：10時間20分10秒

730.0ページ目：198.49330043792725秒
総取得件数：73000
完了：77.068%
残り時間：10時間17分29秒

731.0ページ目：164.31692028045654秒
総取得件数：73100
完了：77.174%
残り時間：10時間14分37秒

732.0ページ目：169.2856047153473秒
総取得件数：73200
完了：77.28%
残り時間：10時間11分47秒

733.0ページ目：172.826597213745

839.0ページ目：159.2720046043396秒
総取得件数：83900
完了：88.576%
残り時間：5時間7分2秒

840.0ページ目：166.1543025970459秒
総取得件数：84000
完了：88.681%
残り時間：5時間4分12秒

841.0ページ目：159.07480239868164秒
総取得件数：84100
完了：88.787%
残り時間：5時間1分22秒

842.0ページ目：153.99855041503906秒
総取得件数：84200
完了：88.893%
残り時間：4時間58分31秒

843.0ページ目：168.26668119430542秒
総取得件数：84300
完了：88.998%
残り時間：4時間55分42秒

844.0ページ目：167.77442407608032秒
総取得件数：84400
完了：89.104%
残り時間：4時間52分53秒

845.0ページ目：164.69978213310242秒
総取得件数：84500
完了：89.209%
残り時間：4時間50分4秒

846.0ページ目：170.14504623413086秒
総取得件数：84600
完了：89.315%
残り時間：4時間47分15秒

847.0ページ目：159.38459849357605秒
総取得件数：84700
完了：89.421%
残り時間：4時間44分25秒

848.0ページ目：162.2518048286438秒
総取得件数：84800
完了：89.526%
残り時間：4時間41分35秒

849.0ページ目：160.42399096488953秒
総取得件数：84900
完了：89.632%
残り時間：4時間38分45秒

850.0ページ目：171.9725480079651秒
総取得件数：85000
完了：89.737%
残り時間：4時間35分56秒

851.0ページ目：150.40997672080994秒
総取得件数：85100
完了：89.843%
残り時間：4時間33分5秒

852.0ページ目：164.30733823776245秒
総取得件数：85200
完了：89.948%
残り時間：4時間30分16秒

853.0ページ目：151.223450422287秒
総取得件数：85300
完了：9

In [2]:
# 地価情報を拾う

url = "https://tochidai.info/tokyo23/"

# ページ読み込み関数（リトライ機能付き）
@retry(tries=3, delay=10, backoff=2)
def load_page(load_url):
    html = requests.get(load_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

soup = load_page(url)

In [58]:
ranking = soup.find('tbody')

In [57]:
# 地価テーブルを作った

rank_name = ranking.find_all('a')
rank_price = ranking.find_all(class_="tsubo-price c_tb")

land_price_ranking = {}
for name, price in zip(rank_name,rank_price):
    land_price_ranking[name.text] = price.text
    land_price_ranking[name.text] = re.search('([0-9]+)万([0-9]+)円/坪',land_price_ranking[name.text]).groups()
    land_price_ranking[name.text] = int(land_price_ranking[name.text][0] + land_price_ranking[name.text][1])

In [55]:
table_land_price = pd.DataFrame(np.c_[list(land_price_ranking.keys()),list(land_price_ranking.values())], columns={'区','地価'})

In [56]:
table_land_price.to_csv('./data/csv/land_price.csv')